
# Design Optimization: HW4

## Seyed Yousef Soltanian 
 ### you can find the file at https://github.com/YousefSoltanian/MAE598_Design_Optimization.git

### Problem 1:

In [289]:
import numpy as np

### defining the objective function and the gradient of objective function
def objective(x):
    # Calculate the objective function f
    f = x[0]**2+(x[1]-3)**2;
    
    return f

def objectiveg(x):
    # Calculate the gradient of the objective (row vector)
    # df = [df/dx1, df/dx2, ..., df/xn]
    
    df = np.array([2*x[0], 2*x[1]-6])
    return df

## test: 

print(objectiveg(np.array([1,1])))
kk = objectiveg(np.array([1,1]))
print(np.shape(kk))

[ 2 -4]
(2,)


In [307]:
def constraint(x):
    return np.array([x[1]**2 - 2*x[0], (x[1] - 1)**2 + 5*x[0] - 15])
## test:
x=np.array([[1],[1]])
print(constraint(x).shape)


(2, 1)


In [331]:
def constraintg(x):
    # Calculate the gradient of the constraints
    # dg = [dg1/dx1, dg1/dx2, ... , dg1/dxn;
    #       dg2/dx1, dg2/dx2, ... , dg2/dxn;
    #       ...
    #       dgm/dx1, dgm/dx2, ... , dgm/dxn]
    dg = np.array([[-2, 2*x[1][0]],[5, 2*x[1][0]-2]])
    return dg

## test:
x=np.array([[1],[1]])
kk=constraintg(x)

print(kk@x)


[[0]
 [5]]


In [281]:
import numpy as np

def lineSearch(f, df, g, dg, x, s, mu_old, w_old):
    t = 0.1  # scale factor on current gradient: [0.01, 0.3]
    b = 0.8  # scale factor on backtracking: [0.1, 0.8]
    a = 1  # maximum step length
    
    D = s  # direction for x
    
    # Calculate weights in the merit function using equation (7.77)
    w = np.maximum(np.abs(mu_old), 0.5 * (w_old + np.abs(mu_old)))
    
    # terminate if line search takes too long
    count = 0
    while count < 100:
        # Calculate phi(alpha) using merit function in (7.76)
        phi_a = f(x + a * D) + np.dot(w, np.abs(np.minimum(0, -g(x + a * D))))
        
        # Calculate psi(alpha) in the line search using phi(alpha)
        phi0 = f(x) + np.dot(w, np.abs(np.minimum(0, -g(x))))  # phi(0)
        dphi0 = np.dot(df(x), D) + np.dot(w, (dg(x) * D) * (g(x) > 0))  # phi'(0)
        psi_a = phi0 + t * a * dphi0  # psi(alpha)
        
        # stop if condition satisfied
        if phi_a < psi_a:
            break
        else:
            # backtracking
            a = a * b
            count = count + 1
    
    return a, w


In [338]:
import numpy as np
from scipy.optimize import minimize


def mysqp(f, df, g, dg, x0, opt):
    # Set initial conditions
    x = x0  # Set the current solution to the initial guess

    # Initialize a structure to record the search process
    solution = {'x': []}
    solution['x'].append(x.copy())  # Save the initial solution

    # Initialization of the Hessian matrix
    W = np.eye(len(x))  # Start with an identity Hessian matrix

    # Initialization of the Lagrange multipliers
    mu_old = np.zeros(len(g(x)))  # Start with zero Lagrange multiplier estimates

    # Initialization of the weights in the merit function
    w = np.zeros(len(g(x)))  # Start with zero weights

    # Set the termination criterion
    gnorm = np.linalg.norm(df(x) + mu_old @ dg(x))  # norm of Lagrangian gradient

    while gnorm > opt['eps']:  # if not terminated
        # Implement QP problem and solve
        if opt['alg'] == 'myqp':
            # Solve the QP subproblem to find s and mu (using your own method)
            s, mu_new = solveqp(x, W, df, g, dg)
        else:
            # Solve the QP subproblem to find s and mu (using MATLAB's solver)
            qpalg = {'Algorithm': 'active-set', 'Display': 'off'}
            result = minimize(
                lambda s: (0.5 * s.T @ W @ s) + (df(x)[0]*s[0]+df(x)[1]*s[1]) ,
                np.zeros_like(x).flatten(),
                constraints={'type': 'ineq', 'fun': lambda s: g(x) + dg(x) @ s},
                options={'disp': False},
            )
            s = result.s
            mu_new = result['ineq']
        
        # opt['linesearch'] switches line search on or off.
        # You can first set the variable "a" to different constant values and see how it
        # affects the convergence.
        if opt['linesearch']:
            a, w = lineSearch(f, df, g, dg, x, s, mu_old, w)
        else:
            a = 0.1

        # Update the current solution using the step
        dx = a * s  # Step for x
        x = x + dx  # Update x using the step

        # Update Hessian using BFGS. Use equations (7.36), (7.73), and (7.74)
        # Compute y_k
        y_k = (
            df(x) + mu_new @ dg(x) - df(x - dx) - mu_new @ dg(x - dx)
        ).reshape((-1, 1))
        # Compute theta
        if dx @ y_k >= 0.2 * dx @ W @ dx:
            theta = 1
        else:
            theta = (0.8 * dx @ W @ dx) / (dx @ W @ dx - dx @ y_k)
        # Compute dg_k
        dg_k = theta * y_k + (1 - theta) * W @ dx
        # Compute new Hessian
        W = (
            W
            + np.outer(dg_k, dg_k) / (dg_k @ dx)
            - np.outer(W @ dx, W @ dx) / (dx @ W @ dx)
        )

        # Update termination criterion
        gnorm = np.linalg.norm(df(x) + mu_new @ dg(x))  # norm of Lagrangian gradient
        mu_old = mu_new

        # Save current solution to solution['x']
        solution['x'].append(x.copy())

    return solution


In [339]:
#%%%%%%%%%%%%%% Main Entrance %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#% Instruction: Please read through the code and fill in blanks 
#% (marked by ***). Note that you need to do so for every involved
#% function, i.e., m files. 

#%% Optional overhead


#%% Optimization settings
#% Here we specify the objective function by giving the function handle to a
#% variable, for example:
f = objective; # replace with your objective function
# In the same way, we also provide the gradient of the 
# objective:
df = objectiveg; 

g = constraint;
dg = constraintg;

# Note that explicit gradient and Hessian information is only optional.
# However, providing these information to the search algorithm will save
# computational cost from finite difference calculations for them.

# Specify algorithm
opt = {}
opt['alg'] = 'pyqp'  # 'myqp' or 'matlabqp'

# Turn on or off line search. You could turn on line search once other
# parts of the program are debugged.
opt['linesearch'] = True  # False or True

# Set the tolerance to be used as a termination criterion:
opt['eps'] = 1e-3

# Set the initial guess:
x0=np.array([[1],[1]])

# Feasibility check for the initial point.
if max(g(x0)>0):
    errordlg('Infeasible intial point! You need to start from a feasible one!');


# Run optimization
# Run your implementation of SQP algorithm. See mysqp.m
solution = mysqp(f, df, g, dg, x0, opt);


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

### Problem 2:

For this Problem we have to solve the following optimization problem:

$$
Minimize_x: \
f(x) = \sum_{t=0}^{t=T} a(t)
$$
$$
Subjected \ to: h(T) = 0 \ , \ v(T) = 0
$$

so we can form the Hamiltonian based on the state space equations:

$$
H = \lambda^Tf - e = \begin{bmatrix} \lambda_1 & \lambda_2 & \lambda3 \end{bmatrix}\begin{bmatrix}v \\ -g-\frac{a}{m} \\ -Ka\end{bmatrix} \ = \frac{\lambda_1v + \lambda_2(-g+\frac{a}{m}) - \lambda_3Ka - a}{a(\frac{\lambda_2}{m} - \lambda_3K - 1) + \lambda_1v - \lambda_2g}
$$

first we try to find all the $\lambda_is$:

$$
 -\frac{\partial{H}}{\partial{h}} = \dot{\lambda_1} \rightarrow \dot{\lambda_1}  = 0 \rightarrow \lambda_1 = c_0 \\
 -\frac{\partial{H}}{\partial{v}} = \dot{\lambda_2} \rightarrow \dot{\lambda_2}  = -\lambda_1 \rightarrow \lambda_2 = c_0t +c1 \\
 -\frac{\partial{H}}{\partial{m}} = \dot{\lambda_3}  \rightarrow \dot{\lambda_3} = \lambda_2\frac{a}{m^2} \rightarrow \frac{(0.5c_0t^2 +c_t1)a}{m^2}+c2\\
$$

now, based on the Hamiltonian, we have to chose $a$ in a way to maximize H , by looking at H we can see that the desired a(t) is as:

$$
a = 1 \ if \ \frac{\lambda_2}{m} - \lambda_3K - 1 > 0 \\
a = 0 \ if \ \frac{\lambda_2}{m} - \lambda_3K - 1 > 0
$$

now to get the explicit needed policy, we have to put $\lambda_2$ and $\lambda_3$ back to the above equation. To find the constants $c_0,c_1,c_2$ we have to use the boundary conditions $h_0,v_0,t_0$ and $h*,v*$. we basically need to use the obtained equation for a and put it back into our states equations, then using those equations chose each $c_i$ in a way that all the boundaries conditions get satisfied, especially the final one.